# MoCO - Momentum Contrastum - https://arxiv.org/abs/1911.05722v3

MoCo, or Momentum Contrast, jest kontrastową metodą self-supervised learning.

Jako, że MoCo jest metodą kontrastową to naszym zadaniem będzie maksymalizacja podobieństwa pomiędzy pozytywnymi parami, a negatywnymi parami.

Do zmierzenia prawdopodobieństwa pomiędzy parami często używa się InfoNCE, które używa iloczynu skalarnego do wyliczenia prawdopodobieństwa. Poniżej znaduje się wzór tej funkcji:

<img src="notebook_images/InfoNCE_loss.png" alt="drawing" width="400"/>

Powyższy koszt można też zrozumieć jako log loss softmax-based klasyfikatora, które próbuje sklasyfikować próbkę jako klasę pozytywną.

Co więc wyróżnia MoCO od innych metod kontrastowych?

Zdaniem twórców MoCo w metodach kontrastowych kluczem jest zbudowanie bardzo dużej bazy obrazów tak, żeby można było łatwo generować pozytywne i negatywne pary i na bazie nich wytrenować odpowiedni enkoder.

Autorzy proponują następujące rozwiązanie:
1. Zapisywanie użytych już obrazów w tzw. "Dictionary", które strukturalnie będzie kolejką. Dzięki temu podczas brania kolejnego batcha obrazów będziemy mieli bardzo dużo negatywnych przykładów
2. Rozmiar naszej kolejki może być znacznie wyższy od rozmiaru batcha. Zalecane jest jednak ustawienie określonego rozmiaru i usuwanie najstarszego batcha. Wynika to z tego, że najstarszy batch jest najmniej aktualny ze wszystkich zapisanych batchy
3. Używanie momentum update do aktualizacji wag key encodera. Aktualizacja wag została przedstawiona poniższym wzorem

<img src="notebook_images/momentum_update.png" alt="drawing" width="400"/>

4. Po wytrenowaniu enkodera dalej bierzemy tylko query encoder

Działanie algorytmu zostało dokładnie przedstawione na poniższym obrazku

<img src="notebook_images/Moco_desc.png" alt="drawing" width="600"/>

# Import bibliotek

In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from tqdm import tqdm
from src.dataset import *
from src.early_stopping import *
from src.self_supervised_modules import *
from src.utils import *

# Wczytanie danych do trenowania końcowego klasyfikatora

In [ ]:
data_train = np.load('data_transform/subset0/data.npz')
data_train

In [ ]:
X_train = data_train['images']
X_train.shape

In [ ]:
y_train = data_train['labels']
y_train.shape

# Wczytanie danych do trenowania enkodera

In [ ]:
images1 = np.load('data_transform/subset1/data.npz')['images']
images2 = np.load('data_transform/subset2/data.npz')['images']
images3 = np.load('data_transform/subset3/data.npz')['images']

all_images = np.concatenate((images1, images2, images3))
all_images.shape

# Wczytanie danych do walidacji klasyfikatora

In [ ]:
data_val = images4 = np.load('data_transform/subset4/data.npz')
data_val

In [ ]:
X_val = data_val['images']
X_val.shape

In [ ]:
y_val = data_val['labels']
y_val.shape

# Definicja augmentacji
Ponieważ nasz zbiór danych to 10 najważniejszych obrazów to augmentacje, które zastosujemy muszą być identyczne dla każdego obrazu, żeby nie zatracić żadnej informacji.

Pomysły na augmentację:
1. Horizontal i Vertical flip obraza
2. Rotacja

Trudno wprowadzić więcej augmentacji gdyż ryzykujemy wówczas zbytnie zmienienie obrazów

In [ ]:
def apply_transformations(images, chance_flip_horizontal=0.5, chance_flip_vertical=0.5, chance_rotate=0.5, rotate_angle_max=45, rotate_angle_min=-45):
    batch_size = images.shape[0]
    no_images = images.shape[2]

    for i in range(batch_size):
        flip_horizontal = random.random() < chance_flip_horizontal
        flip_vertical = random.random() < chance_flip_vertical
        rotate = random.random() < chance_rotate
        if rotate:
            rorate_angle = random.randint(rotate_angle_min, rotate_angle_max)
        
        for j in range(no_images):
            image_augment = images[i, 0, j]
            if flip_horizontal:
                image_augment = torchvision.transforms.functional.hflip(image_augment)
            if flip_vertical:
                image_augment = torchvision.transforms.functional.vflip(image_augment)
            if rotate:
                image_augment = image_augment.unsqueeze(0)
                image_augment = torchvision.transforms.functional.rotate(image_augment, rorate_angle, fill=-1000)
                image_augment = image_augment.squeeze()

            images[i, 0, j] = image_augment

    return images

# Definicja MoCO

In [ ]:
class MoCo(nn.Module):
    """
    Based on offical implementaion
    https://github.com/facebookresearch/moco/blob/3631be074a0a14ab85c206631729fe035e54b525/moco/builder.py#L6
    """
    def __init__(self, encoder, queue_size=4096, momentum=0.999, temperature=0.07):
        super(MoCo, self).__init__()

        self.queue_size = queue_size
        self.momentum = momentum
        self.temperature = temperature

        self.encoder_query = copy.deepcopy(encoder)
        self.encoder_key = copy.deepcopy(encoder)

        # Create the queue and register it so it won't update during optimalisation
        self.register_buffer("queue", torch.randn(encoder.out_size, queue_size))
        self.queue = nn.functional.normalize(self.queue, dim=0)

        self.register_buffer("queue_ptr", torch.zeros(1, dtype=torch.long))

    @torch.no_grad()
    def momentum_update_key_encoder(self):
        """
        Momentum update of the key encoder
        """
        for param_query, param_key in zip(self.encoder_query.parameters(), self.encoder_key.parameters()):
            param_key.data = param_key.data * self.momentum + param_query.data * (1. - self.momentum)
        
    @torch.no_grad()
    def _dequeue_and_enqueue(self, keys):
        batch_size = keys.shape[0]

        ptr = int(self.queue_ptr)

        # enqueue and dequeue at the same time (transpose is needed)
        self.queue[:, ptr:ptr + batch_size] = keys.T
        
        # move pointer to the next oldest batch
        ptr = (ptr + batch_size) % self.queue_size

        # update saved queue pointer
        self.queue_ptr[0] = ptr

    def forward(self, im_query, im_key):
        query = self.encoder_query(im_query) 
        query = nn.functional.normalize(query, dim=1)

        with torch.no_grad():
            key = self.encoder_key(im_key)
            key = nn.functional.normalize(key, dim=1)

        # positive logits: Nx1
        l_pos = torch.bmm(query.unsqueeze(1), key.unsqueeze(2)).squeeze(-1)
        
        # negative logits: NxK
        l_neg = torch.mm(query, self.queue.clone().detach())
        
        # logits: Nx(1+K)
        logits = torch.cat([l_pos, l_neg], dim=1)

        # apply temperature
        logits /= self.temperature

        # labels: positive key indicators
        labels = torch.zeros(logits.shape[0], dtype=torch.long).cuda()

        # dequeue and enqueue (operation with no gradient update so it can be done any time)
        self._dequeue_and_enqueue(key)

        return logits, labels

# Trenowanie enkodera używanego w MoCO

In [ ]:
def train_moco(model, optimiser, dataloader, loss_fn, no_epochs=100, augment_params=None):
    if augment_params is None:
        augment_params = {}

    losses = []
    for epoch in range(no_epochs):
        sum_loss = 0
        for images in tqdm(dataloader):
            x_query = apply_transformations(images, **augment_params).cuda()
            x_key = apply_transformations(images, **augment_params).cuda()
    
            logits, labels = model(x_query, x_key)
    
            loss = loss_fn(logits, labels)
    
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
    
            model.momentum_update_key_encoder()

            sum_loss += loss.item()
            
        print(f"Epoch {epoch}: loss = {sum_loss:.3f}")
        losses.append(sum_loss)
        
    plt.plot(losses)
    plt.title("Loss", fontsize=18)
    plt.xlabel("Iteration", fontsize=14)
    plt.ylabel("Loss", fontsize=14)
    plt.show()

In [ ]:
seed_everything()

# 100 - default, 3 - for testing the code
no_epochs = 100
# no_epochs = 3

unlabelled_dataset = UnlabelledDataset(all_images)
unlabelled_dataloader = DataLoader(unlabelled_dataset, batch_size=256, shuffle=True, drop_last=True)

encoder = Encoder()
moco = MoCo(encoder).cuda()
optimiser = optim.SGD(moco.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)
loss_fn = nn.CrossEntropyLoss()

train_moco(moco, optimiser, unlabelled_dataloader, loss_fn, no_epochs)

# Trenowanie klasyfikatora

In [ ]:
seed_everything()

train_dataset = LabelledDataset(X_train, y_train)
val_dataset = LabelledDataset(X_val, y_val)

train_dl = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=64, shuffle=True)

model = SelfSupervisedClassifier(moco.encoder_query, 2).cuda()
optimiser = optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

fit_classifier(
    model=model, optimiser=optimiser, loss_fn=loss_fn,
    train_dl=train_dl, val_dl=val_dl, epochs=50, early_stop=EarlyStopping(model_dir='model/moco', patience=5), print_metrics=True
)

# Liczenie metryk klasyfikacyjnych

In [ ]:
val_dataset = LabelledDataset(X_val, y_val)
val_dl = DataLoader(val_dataset, batch_size=64, shuffle=True)
calculate_metrics(model, val_dl)